# ENTSO-G Daten

Die Daten der European Network of Transmission System Operators for Gas (ENTSO-G) werden auf dem Transparenzportal veröffentlicht.

Über eine API lassen sich die Daten, sowie Historiendaten abfragen.

Das Projekt unterteilt sich zunächst in 3 Teile:

1. Daten abfragen und speichern
2. Daten aufbereiten
3. Daten visualisieren

Im Gegensatz zu den ENTSO-E Daten gibt es hierbei noch keinerlei Web-Frontends, welche die Daten visualisieren

### 0. Definitionen

* Bidding zones sind die Marktgebiete. Um zwischen Marktgebieten Gas zu transferieren müssen die Gas-Pipelines gebucht werden. 
* Jedes Land hat sein eigenes Marktgebiet (mit Ausnahmen, bspw hat DE bis 2021 noch 2 [NCG und GASPOOL])
* In einem Marktgebiet gibt es verschiedene Operatoren, welche das Gasnetz betreiben
* Es gibt L-Gas und H-Gas, welche unterschiedlich gebucht und transferiert werden
* Jede Pipeline hat einen Entry und einen Exit. Gas fließt immer in eine Richtung


So gibt es bspw. die Pipeline "Nord Stream"

## Datenbeschaffung

Die Daten können über die API in CSV, JSON oder XML runtergeladen werden.

Tabellarische Daten lassen sich davon am effizientesten als CSV darstellen, weshalb dieses Format gewählt wurde.

Diese werden anschließend in eine Parquet-Datei gespeichert.
Darüber können wir nun über die Historien-Daten zugreifen.

Die Daten der ENTSO-G stehen in diesem Format über die API seit 2017 zur Verfügung.

In [ ]:
import pandas as pd
aggData = pd.read_csv('https://transparency.entsog.eu/api/v1/AggregatedData.csv?limit=1000')
interconn = pd.read_csv('https://transparency.entsog.eu/api/v1/Interconnections.csv?limit=-1')
# the connectionpoints contain invalid data, as each line ends with an extra seperator
conn = pd.read_csv('https://transparency.entsog.eu/api/v1/connectionpoints.csv?limit=-1',index_col=False)

In [ ]:
import requests
csv = requests.get('https://transparency.entsog.eu/api/v1/connectionpoints.csv?limit=-1')
cleaned = csv.text.replace('commercialType,importFromCountryKey','installation,commercialType,importFromCountryKey')
from io import StringIO
fileLike = StringIO(cleaned)
conn = pd.read_csv(fileLike)

In [ ]:
jsn = requests.get('https://transparency.entsog.eu/api/v1/connectionpoints.json?limit=-1')
conn = pd.DataFrame(jsn.json()['connectionpoints'])


Zum Herunterladen der Daten wurde eine Python Klasse geschrieben, welche die Daten für einen gebenen Bereich von der Api lädt. Diese werden anschließend sowohl in einer SQLite Datenbank, wie auch in Parquet-Dateien mithilfe von Spark gespeichert.

Die Datenstruktur sieht wie folgt aus:

![Datenbank-Struktur](Diagrams-ENTSO-G.png)

Um auf die Daten nachfolgend zuzugreifen verwenden wir Spark

In [ ]:
import findspark

findspark.init()
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pandas as pd

conf = SparkConf().setAppName('entsog').setMaster('local')
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

In [ ]:
sparkfolder='data/spark'
sparkfolder='entsog_spark/spark'

In [ ]:
conn.to_parquet(f'{sparkfolder}/connectionpoints')

In [ ]:
interconnections = spark.read.parquet(f"{sparkfolder}/Interconnections.parquet")
balancingzones = spark.read.parquet(f"{sparkfolder}/balancingzones.parquet")
opdata = spark.read.parquet(f"{sparkfolder}/operationaldata")
oppointdirs = spark.read.parquet(f"{sparkfolder}/operatorpointdirections.parquet")
operators = spark.read.parquet(f"{sparkfolder}/operators.parquet")
connectionpoints = spark.read.parquet(f"{sparkfolder}/connectionpoints.parquet")

In [ ]:
# generalRemarks must be deleted as column contains different datatypes

# structure files in parquet, so that hierarchy is better:
%time opdata.drop('generalRemarks').write.partitionBy(['year','month']).parquet(f"{sparkfolder}/operationaldata_cleaned")

In [ ]:
# converting time column to be grouped by a given period works as easy as this:
from pyspark.sql.functions import date_trunc
opdata = ( opdata.withColumn("day", date_trunc('day',"periodFrom"))
          .withColumn("hour", date_trunc('hour',"periodFrom"))
         )

In [ ]:
# selecting only necessary stuff from static tables
inter= interconnections.select(['pointLabel','pointTpMapX','pointTpMapY','fromCountryKey','fromOperatorLabel','fromBzLabel','fromBzLabel','toOperatorLabel','toCountryKey','pointKey','toPointKey','toPointLabel']).toPandas()
balancing= balancingzones.select(['bzLabel','tpMapX','tpMapY']).toPandas()
conn = (connectionpoints.withColumn("long", connectionpoints.tpMapX)
        .withColumn("lat", connectionpoints.tpMapY)
        .select(['pointKey','lat','long','infrastructureLabel']).toPandas()
       )
ops=operators.select(['operatorKey','operatorLabel','operatorCountryKey','operatorTypeLabelLong']).toPandas()

In [ ]:
# show small table of available operators
import plotly.graph_objects as go
fig = go.Figure(data=[go.Table(
    header=dict(values=['OperatorKey','Betreiber Bezeichnung','Land des Betreibers','Art des Betreibers'],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[ops.operatorKey, ops.operatorLabel, ops.operatorCountryKey, ops.operatorTypeLabelLong],
               fill_color='lavender',
               align='left'))
])

fig.show()


Die x und y Koordinaten in den Daten für Interconnections und Bidding zones sind leider nicht in Koordinaten-Form zu bringen. Keine Projektion oder Verdrehung brachte ein ordentliches Bild.

Das Problem wurde anschließend dadurch gelöst, indem die Map-Tile Bilder direkt von dem ENTSO-G Server herunter geladen wurden und das Kartenmaterial nun selbst lokal gehostet wird.

## Kartenmaterial beschaffung
Die Kartendaten laden wir direkt von deren Server runter.
Hier gibt es 5 Zoomstufen, für die jeweils Kartenmaterial als Bilddatei heruntergeladen werden kann.\
Außerdem gibt es verschiedene Overlays für Karten, welche angezeigt werden können.

Leider sind die Daten im TMS-Format, das heißt, dass die Y-Achse invertiert ist.

Für mehr Informationen, siehe hier: https://gist.github.com/tmcw/4954720

Dort ist auch beschrieben, wie man diese Daten in das XYZ konvertieren kann. Das ist notwendig, da Plotly TMS leider [nicht unterstützt](https://github.com/plotly/plotly.py/issues/1610)

In [ ]:
import requests
import os
import entsog_maploader

#shutil.rmtree("data_xyz")

routes = ['countries_zones','pipelines_small_medium_large','pipelines_medium_large','pipelines_large','drilling_platforms','gasfields','projects','country_names']
url = "https://transparency.entsog.eu/assets/images/map_layers/"
#for route in routes:
#    loadMap('data_tms',route,url)
#    convertTmsXyz('data_tms/'+route,'data_xyz/'+route)

#shutil.make_archive('data_xyz', 'zip', 'data_xyz')                    

import inspect
lines = inspect.getsource(entsog_maploader.loadMap)
print(lines)

Man könnte hier nun auch ein GeoJSON erstellen um eine Chloropleth Karte der Gebiete zu erstellen, hierzu müsste man auf https://geojson.io das Kartenmaterial als Layer einbinden und die Polygone der Marktgebiete einzeichnen.

Neben diesem Nebenschauplatz, besteht natürlich auch der Hauptteil, welcher das Herunterladen, strukturieren und abfragen der Daten beinhaltet.

Die Daten können nun über die API herunter geladen werden und in einer Datenbank oder Parquet-Datei abgespeichert werden.

Insgesamt haben die Daten bereits eine hohe Qualität, dennoch gibt es einige Fallstricke die man beachten muss.
So gibt es Pipe-in-Pipe Verbindungen, welche bei Nichtbeachtung eine doppelte Buchung abgeben.

Diese gilt es herauszufiltern.

In [ ]:
layers = []

routes = ['countries_zones','pipelines_small_medium_large','pipelines_medium_large','pipelines_large','drilling_platforms','gasfields','projects','country_names']
routes = ['countries_zones','pipelines_small_medium_large','country_names']

for route in routes:
    layers.append({   "below": 'traces',
                "sourcetype": "raster",
                "sourceattribution": '<a href="https://transparency.entsog.eu/#/map">ENTSO-G Data</a>',
                "source": [
                    "data/mapdata_xyz/"+route+"/{z}/{x}/{y}.png"]})

### Karte der Verbindungen

In [ ]:
import plotly.express as px
#px.colors.qualitative.swatches()

# nur die GrenzFälle
inter_border= inter[inter['fromCountryKey'] !=inter['toCountryKey']]

In [ ]:
import plotly.express as px
fig = px.scatter_mapbox(inter_border, lat="pointTpMapY", lon="pointTpMapX", hover_name="pointLabel", color='fromCountryKey',hover_data=["pointKey", "fromCountryKey",'fromOperatorLabel','toOperatorLabel',"toCountryKey"],
                        zoom=1, height=600)
fig.update_layout(mapbox_style="white-bg",mapbox_layers=layers,margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Karte der Marktgebiete

In [ ]:
import plotly.express as px
#import plotly.graph_objects as go
fig = px.scatter_mapbox(balancing, lat="tpMapY", lon="tpMapX", hover_name="bzLabel", #color='bzLabel',
                        color_discrete_sequence=["fuchsia"], zoom=1, height=600,title='Karte der Marktgebiete')
fig.update_layout(mapbox_style="white-bg",mapbox_layers=layers,margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


### Karte der Verbindungspunkte

In [ ]:
import plotly.express as px
#import plotly.graph_objects as go
fig = px.scatter_mapbox(conn, lat="lat", lon="long", hover_name="pointKey", hover_data=["infrastructureLabel"],color="infrastructureLabel",
                        zoom=1, height=600,title='Karte der Verbindungspunkte')
fig.update_layout(mapbox_style="white-bg",mapbox_layers=layers,margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Auswertungen

Kommen wir nun zurück zu den Gasdatenflüssen, die wir Visualisieren möchten

In [ ]:
beg=opdata.select('periodFrom').rdd.min()
end=opdata.select('periodFrom').rdd.max()

print('Datensequenz von {} bis {}'.format(beg[0],end[0]))

In [ ]:
operators=opdata.select('operatorKey','operatorLabel').distinct().toPandas()

In [ ]:
from pyspark.sql.functions import date_trunc
pdf =(
    opdata.withColumn("day", date_trunc('day',"periodFrom"))
    .withColumn("month", date_trunc('month',"periodFrom"))
    .withColumn("year", date_trunc('year',"periodFrom"))
).select('operatorKey','year','value').groupby(['operatorKey','year']).sum('value')
%time pdf = pdf.toPandas()

In [ ]:
operators.sort_values('operatorKey')
o = (opdata.withColumn("day", date_trunc('day',"periodFrom"))
     .select(['value','day'])
     .filter('operatorKey = "DE-TSO-0013"')
     .groupby('day').sum('value')
    )
%time pdf2=  o.toPandas()
pdf2 = pdf2.sort_values('day')

In [ ]:
%time pdf =(opdata).select('operatorKey','year','value').groupby(['operatorKey','year']).sum('value').toPandas()

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px

#plt.xticks(rotation=45)
#plt.scatter(pdf['day'],pdf['sum(value)'])

px.line(pdf2,x='day',y='sum(value)')

In [ ]:
pdf.pivot(index='year', columns='operatorKey',values='sum(value)').plot()

## Beanspruchung der Pipelines

Auslastung

In [ ]:
%time sp = opdata.select(['year','month','operatorKey','value']).groupby(['operatorKey']).sum('value').toPandas()

In [ ]:
import plotly.express as px

fig = px.scatter(sp, x="operatorKey", y="sum(value)")

fig.update_xaxes(tickangle=45, tickfont=dict(family='Rockwell', color='crimson', size=8))

## Verlauf der Nordstream

Auslastung

Die Nord Stream hat als interne Bezeichnung den pointKey ITP-00120.
Sie fördert Gas von RU nach DE.
Genau genommen zu den PointKeys 

* ITP-00491 (Greifswald)
* ITP-00250 (Greifswald)
* ITP-00251 (Gw-Opal)? Wird hier aus der OPAL eingespeist? Unsinn?
* ITP-00247 (Greifswald)
* ITP-00297 (Greifswald)
* ITP-00454 (Lubmin, reguliert)

Diese Punkte erhalten Gas von der Nordstream und liegen allesamt in Deutschland.

Die BZ erhält man aus den Interconnections, allerdings nur für die (tote) Leitung von RU nach DE: GASPOOL
Die Datenlage mit Punkten außerhalb der EU ist hier sehr schlechter


In [ ]:
inter[(inter.toOperatorLabel=='Nord Stream')|(inter.fromOperatorLabel=='Nord Stream')]

In [ ]:
pips =list(inter[(inter.toOperatorLabel=='Nord Stream')|(inter.fromOperatorLabel=='Nord Stream')].toPointKey)
pips

In [ ]:
%time d= opdata.filter(opdata.pointKey.isin(pips)).groupby(['pointKey','directionKey','day']).sum('value').toPandas()

In [ ]:
silvester = (
    opdata#.select(['year','hour','operatorKey','value','pointKey'])
    .filter('"2019-01-01" <= day and day < "2019-02-01"')
    )
silvester

In [ ]:
%time sil = silvester.filter(silvester.pointKey.isin(pips)).groupby(['pointLabel','directionKey','hour']).sum('value').toPandas()

In [ ]:
import plotly.express as px

sil=sil.sort_values('hour')
fig = px.line(sil, x='hour', y='sum(value)', color='pointLabel', title='Durchsatz der Nordstream Pipeline')
fig.update_layout( xaxis_title='Datum',
                   yaxis_title='Durchsatz in kW gemittelt pro Stunde')
fig.show()

In [ ]:
df = d.pivot(index='day', columns=['pointKey','directionKey'],values='sum(value)')
import plotly.express as px

d=d.sort_values('day')
fig = px.line(d, x='day', y='sum(value)', color='pointKey', title='Durchsatz der Nordstream Pipeline')
fig.update_layout( xaxis_title='Datum',
                   yaxis_title='Durchsatz in kWh/')
fig.show()

In [ ]:
d.pivot(index='day', columns=['pointKey','directionKey'],values='sum(value)').plot()
#vs melt

# Physical Flow Widget

## Abweichungen des Physical Flows zu den Allokationen darstellen

## Flussdiagramm

nach Datum über die Zeit oder als Übergangsmatrix/Sankey-Diagramm

Gruppierung nach Land/Marktgebiet


Visualisieren wo Gas benötigt wird und wo es nur geschickt wird

Visualisieren der Gas-Pipeline (Kartenfehler finden)

## Anteil Gasversorgung eines Operators am Marktgebiet

Erzeugen eines DashBoards der vorhandenen Daten, ähnlich wie Energy-charts.info

In einem Marktgebiet kaufen und verkaufen verschiedene Operatoren Gas.
Pro Marktgebiet kann man jeweils für Käufe und Verkäufe die Menge nach Operator aufschlüsseln